# Notebook 1 — Complete analysis (Populated)

This notebook extends the base analysis with:
- Rolling-window features (7-day and 30-day moving averages and vol)
- Correlation heatmap between daily metrics and the Fear & Greed index
- A short, transparent backtest using buy/sell imbalance signals (next-day PnL as outcome)

Run all cells top-to-bottom. If running in Colab, upload the CSVs or mount Drive and set `DATA_DIR`.


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import display

plt.rcParams['figure.figsize'] = (12,5)
plt.rcParams['figure.dpi'] = 100

DATA_DIR = 'C:\Users\hulkh\Downloads\ds_Siva_Venkata_Bhanu_Prakash\csv_files'
print('DATA_DIR =', DATA_DIR)


## Load (or compute) daily aggregates
This cell prefers an existing `/mnt/data/daily_aggregates.csv`. If it's not present it will try to compute daily aggregates from `historical_data.csv`.

In [ ]:
daily_path = os.path.join(DATA_DIR, 'daily_aggregates.csv')
hist_path = os.path.join(DATA_DIR, 'historical_data.csv')
fg_path = os.path.join(DATA_DIR, 'fear_greed_index.csv')

if os.path.exists(daily_path):
    daily = pd.read_csv(daily_path, parse_dates=['date'])
    print('Loaded existing daily_aggregates.csv')
else:
    assert os.path.exists(hist_path), 'historical_data.csv not found to compute daily aggregates.'
    hist = pd.read_csv(hist_path)
    # Attempt to detect columns
    time_col = next((c for c in ['time','Time','date','Date','timestamp','datetime','created_at'] if c in hist.columns), None)
    if time_col is None:
        time_col = next((c for c in hist.columns if 'time' in c.lower() or 'date' in c.lower()), None)
    pnl_col = next((c for c in hist.columns if 'pnl' in c.lower()), None)
    size_col = next((c for c in hist.columns if c.lower() in ['size','qty','quantity','volume']), None)
    if size_col is None:
        size_col = next((c for c in hist.columns if 'size' in c.lower() or 'qty' in c.lower() or 'volume' in c.lower()), None)
    side_col = next((c for c in hist.columns if c.lower() in ['side','Side','buy_sell','direction','trade_type']), None)
    if side_col is None:
        side_col = next((c for c in hist.columns if 'side' in c.lower() or 'buy' in c.lower() or 'sell' in c.lower()), None)
    # parse
    hist['time_parsed'] = pd.to_datetime(hist[time_col], errors='coerce')
    hist['date'] = hist['time_parsed'].dt.date
    if pnl_col is None:
        pnl_col = next((c for c in hist.columns if 'pnl' in c.lower()), None)
    hist['closedPnL_clean'] = pd.to_numeric(hist[pnl_col], errors='coerce').fillna(0)
    hist['size_clean'] = pd.to_numeric(hist[size_col], errors='coerce').fillna(0).abs()
    if side_col is None:
        hist['side_clean'] = hist['size_clean'].apply(lambda x: 'buy' if x>=0 else 'sell')
    else:
        hist['side_clean'] = hist[side_col].astype(str).str.lower().str.strip()
    daily = hist.groupby('date').agg(
        daily_closedPnL = ('closedPnL_clean', 'sum'),
        daily_volume = ('size_clean', 'sum'),
        buy_trades = ('side_clean', lambda s: (s=='buy').sum()),
        sell_trades = ('side_clean', lambda s: (s=='sell').sum()),
        total_trades = ('side_clean', 'count')
    ).reset_index()
    daily['date'] = pd.to_datetime(daily['date'])
    daily = daily.sort_values('date')
    daily.to_csv(daily_path, index=False)
    print('Computed and saved daily_aggregates.csv')

display(daily.head())
print('\nDaily rows:', len(daily))


## Rolling-window features (7-day and 30-day)
Compute moving averages and volatilities for `daily_closedPnL` and `daily_volume`, and create a buy imbalance metric.

In [ ]:
daily = daily.sort_values('date').reset_index(drop=True)
daily['pnl_ma7'] = daily['daily_closedPnL'].rolling(window=7, min_periods=1).mean()
daily['pnl_ma30'] = daily['daily_closedPnL'].rolling(window=30, min_periods=1).mean()
daily['pnl_vol7'] = daily['daily_closedPnL'].rolling(window=7, min_periods=1).std()
daily['vol_ma7'] = daily['daily_volume'].rolling(window=7, min_periods=1).mean()
daily['vol_ma30'] = daily['daily_volume'].rolling(window=30, min_periods=1).mean()
daily['buy_imbalance'] = (daily['buy_trades'] - daily['sell_trades']) / daily['total_trades'].replace(0, np.nan)
daily['buy_imbalance'] = daily['buy_imbalance'].fillna(0)

display(daily[['date','daily_closedPnL','pnl_ma7','pnl_ma30','daily_volume','vol_ma7','buy_imbalance']].head(15))

# Save rolling plots
plt.figure()
plt.plot(daily['date'], daily['daily_closedPnL'])
plt.plot(daily['date'], daily['pnl_ma7'])
plt.plot(daily['date'], daily['pnl_ma30'])
plt.title('Daily Closed PnL with 7d & 30d MA')
plt.xlabel('Date')
plt.ylabel('Closed PnL')
plt.grid(True)
plt.tight_layout()
pnl_ma_png = os.path.join(DATA_DIR, 'rolling_closed_pnl_ma.png')
plt.savefig(pnl_ma_png)
plt.show()

plt.figure()
plt.plot(daily['date'], daily['daily_volume'])
plt.plot(daily['date'], daily['vol_ma7'])
plt.plot(daily['date'], daily['vol_ma30'])
plt.title('Daily Volume with 7d & 30d MA')
plt.xlabel('Date')
plt.ylabel('Volume')
plt.grid(True)
plt.tight_layout()
vol_ma_png = os.path.join(DATA_DIR, 'rolling_volume_ma.png')
plt.savefig(vol_ma_png)
plt.show()


## Correlation heatmap
Build a correlation matrix between `daily_closedPnL`, `daily_volume`, `buy_imbalance`, and (if present) the Fear & Greed index.

In [ ]:
cols = ['daily_closedPnL','daily_volume','buy_imbalance']
if os.path.exists(fg_path):
    fg = pd.read_csv(fg_path)
    fg_date = next((c for c in fg.columns if 'date' in c.lower()), None)
    fg_val = next((c for c in fg.columns if 'value' in c.lower() or 'index' in c.lower() or 'score' in c.lower()), None)
    if fg_date is not None and fg_val is not None:
        fg['date'] = pd.to_datetime(fg[fg_date], errors='coerce').dt.date
        fg_daily = fg[[ 'date', fg_val ]].copy()
        fg_daily.columns = ['date','fear_greed']
        fg_daily['date'] = pd.to_datetime(fg_daily['date'])
        merged = pd.merge(daily, fg_daily, on='date', how='left')
        cols.append('fear_greed')
    else:
        merged = daily.copy()
else:
    merged = daily.copy()

corr_df = merged[cols].corr()
print('Correlation matrix:')
display(corr_df)

# Plot heatmap using matplotlib only
plt.figure(figsize=(6,5))
plt.imshow(corr_df.values, aspect='auto')
plt.colorbar()
plt.xticks(ticks=range(len(cols)), labels=cols, rotation=45, ha='right')
plt.yticks(ticks=range(len(cols)), labels=cols)
plt.title('Correlation heatmap')
heatmap_png = os.path.join(DATA_DIR, 'correlation_heatmap.png')
plt.tight_layout()
plt.savefig(heatmap_png)
plt.show()


## Brief backtest using buy/sell imbalance
Signal definition (simple & transparent):
- `buy_imbalance = (buy_trades - sell_trades) / total_trades`.
- Long signal (1) when `buy_imbalance > 0.10` and `daily_volume > vol_ma7`.
- Short signal (-1) when `buy_imbalance < -0.10` and `daily_volume > vol_ma7`.

We use **next-day closed PnL** as the outcome for the signal (this is a coarse proxy; in a real backtest you would use trade-level fills/prices).

In [ ]:
df = merged.sort_values('date').reset_index(drop=True).copy()
df['signal'] = 0
df.loc[(df['buy_imbalance'] > 0.10) & (df['daily_volume'] > df['vol_ma7']), 'signal'] = 1
df.loc[(df['buy_imbalance'] < -0.10) & (df['daily_volume'] > df['vol_ma7']), 'signal'] = -1

df['next_day_pnl'] = df['daily_closedPnL'].shift(-1)
df['strategy_pnl'] = df['signal'] * df['next_day_pnl']
df['strategy_cum'] = df['strategy_pnl'].cumsum().fillna(method='ffill').fillna(0)

total_pnl = df['strategy_pnl'].sum(skipna=True)
num_trades = (df['signal']!=0).sum()
avg_pnl_per_trade = df.loc[df['signal']!=0, 'strategy_pnl'].mean()
win_rate = (df.loc[df['signal']!=0, 'strategy_pnl'] > 0).mean()
sharpe = None
if df['strategy_pnl'].std() != 0:
    sharpe = (df['strategy_pnl'].mean() / df['strategy_pnl'].std()) * (252**0.5)

print('Backtest summary (simple):')
print('Total PnL (sum of signal * next_day_pnl):', total_pnl)
print('Number of signals:', num_trades)
print('Avg PnL per signal:', avg_pnl_per_trade)
print('Win rate:', win_rate)
print('Sharpe (ann. approx):', sharpe)

plt.figure()
plt.plot(df['date'], df['strategy_cum'])
plt.title('Strategy cumulative PnL (signal * next-day PnL)')
plt.xlabel('Date')
plt.ylabel('Cumulative PnL')
plt.grid(True)
strategy_png = os.path.join(DATA_DIR, 'strategy_cumulative_pnl.png')
plt.tight_layout()
plt.savefig(strategy_png)
plt.show()

signals_path = os.path.join(DATA_DIR, 'daily_signals_backtest.csv')
df.to_csv(signals_path, index=False)
print('\nSaved signals and results to', signals_path)


## Save all artifacts and final notes
The notebook saved plots and CSVs to `/mnt/data/` so you can download them. Remember this backtest is intentionally simple and uses next-day realized PnL as the outcome — for production you would use precise fills/prices and transaction costs.

In [ ]:
artifacts = [pnl_ma_png, vol_ma_png, heatmap_png, strategy_png, signals_path, daily_path]
for a in artifacts:
    print('-', a, 'exists?', os.path.exists(a))

print('\nNotebook generation completed at', datetime.now().isoformat())


----
### End of populated notebook

If you'd like additional expansions (transaction-cost-aware backtest, more sophisticated signals, intraday analysis, or automated report generation), tell me which one and I'll extend the notebook further.